## Overview

We will be inserting an ESP, connected to a sensor and small battery in a glove. This ESP will communicate to another ESP over ESP-now protocol, which will communicate to our arduino over serial connections, and tell it to move accordingly.

Furthermore, I needed a 2 second delay before intializing the serial

### Sensor
We'll be using a MPU5060 sensor which measures acceleration in 3 dimensions, and angular rotation in 3 dimensions.

We send over ESP-now protocol, which has a simple setup.

### Reading the outputs

To read the outputs from our MPU9250 we must read the raw-bytes. I am not exactly sure how it worked, but basically, we imported ```wire.h```, and then wrote to the following parts of memory:
```c++
// MPU9250 and AK8963 registers
#define MPU9250_ADDR              0x68
#define PWR_MGMT_1               0x6B
#define INT_PIN_CFG              0x37
#define ACCEL_XOUT_H             0x3B
#define GYRO_XOUT_H              0x43
#define AK8963_ADDR              0x0C
#define AK8963_CNTL              0x0A
#define AK8963_XOUT_L            0x03

```

then, we could call like:

```c++
Wire.beginTransmission(AK8963_ADDR);
Wire.write(AK8963_XOUT_L);
Wire.endTransmission(false);
Wire.requestFrom(AK8963_ADDR, 6, true);

```

which gives us the data. Also, we would do this a couple times beforehand to calibrate...

### Other things...

However, an important part is that we must be in synch. I made a huge mistake here by not doing so; if we run a loop that goes every 4 microseconds (arduino max speed), this will outpace our 250hz, and be slow.

Therefore, I just added a timer that ensured that the ```loop_begin - current_time``` was at least 1/250th of a second.

Other things i found:

- Divide output of accel by 16384.0 for 'g's
- gyroscope by 131.0 for deg/s

Furthermore, it is necessary to be careful with using the correct datatypes:

- floats for decimal precision
-  int32_t/int16_t for raw-values 

and a big problem I solved was ***Converthing values to int16_t first when reading, and then doing sum = (int32_t)int16_tval!***

## Integrating

To calculate the angle, given angular acceleration in X,Y, and Z dimensions, we must integrate the velocity over time, which involves converting acceleration to velocity:

$$
\sum_{}^{}\frac{dangle}{dt}*\Delta t=\sim angle
$$

but for acceleration, we need to go from acceleration to position. This involves integrating twice, and keeping track of the running-velocity.


## Pitch and yaw values.